# Sistema Integrado de Processamento de Imagens

**Módulo Completo:** Análise de Conversão + Transformações de Contraste + Análise de Filtros de Suavização

Este notebook combina:
- Análise de Métodos de Conversão (Padrão, Média, Canais RGB)
- Conversão para escala de cinza e transformações (Logaritmo, Quadrado, Exponencial)
- Análise de filtros de suavização (Gaussiano, Mediano, Bilateral)
- Inversão de cores e exibição de canais RGB
- Histogramas e métricas de qualidade
- Download em lote

# Célula 1: Importação das Bibliotecas

In [ ]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output, HTML
import io
import zipfile
import os

# Célula 2: Definição de Todas as Funções

In [ ]:
def converter_para_cinza(img_bgr):
    return cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

def aplicar_transformacao(img_cinza, metodo):
    img_float = img_cinza.astype(np.float64)

    if metodo == 'Logaritmo':
        denominador = np.log(1 + np.max(img_float))
        if denominador == 0:
            return img_cinza
        c = 255 / denominador
        img_transformada = c * np.log(1 + img_float)
    elif metodo == 'Quadrado':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 2.0) * 255
    elif metodo == 'Exponencial':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 0.5) * 255
    elif metodo == 'Equalização de Histograma':
        img_transformada = cv.equalizeHist(img_cinza)
    else:
        return img_cinza
    return np.uint8(img_transformada)

def inverter_cores(img_cinza):
    return cv.bitwise_not(img_cinza)

def exibir_canal_cor(img_bgr, canal):
    azul, verde, vermelho = cv.split(img_bgr)
    if canal == 'Vermelho':
        return vermelho
    elif canal == 'Verde':
        return verde
    elif canal == 'Azul':
        return azul
    else:
        return converter_para_cinza(img_bgr)

def plotar_histograma(img_cinza):
    plt.figure(figsize=(6, 4))
    plt.title("Histograma de Tons de Cinza")
    plt.xlabel("Intensidade")
    plt.ylabel("Número de Pixels")
    plt.hist(img_cinza.ravel(), bins=256, range=[0, 256])
    plt.xlim([0, 256])
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def converter_para_cinza_varios_metodos(img_bgr):
    b, g, r = cv.split(img_bgr)
    
    img_cinza_padrao = cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)
    
    img_cinza_media = np.mean(img_bgr, axis=2).astype(np.uint8)
    
    img_canal_r = r
    img_canal_g = g
    img_canal_b = b
    
    metodos = {
        "Padrão": img_cinza_padrao,
        "Média": img_cinza_media,
        "Canal Vermelho": img_canal_r,
        "Canal Verde": img_canal_g,
        "Canal Azul": img_canal_b
    }
    return metodos

def calcular_metricas_conversao(img_cinza):
    contraste_rms = np.std(img_cinza)
    
    imin, imax = np.min(img_cinza), np.max(img_cinza)
    if (imax + imin) == 0:
        contraste_michelson = 0.0
    else:
        contraste_michelson = (float(imax) - float(imin)) / (float(imax) + float(imin))
        
    hist = cv.calcHist([img_cinza], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / hist.sum()
    entropia = -np.sum(hist_norm * np.log2(hist_norm + 1e-6))
    
    return {
        "Contraste RMS": contraste_rms,
        "Contraste Michelson": contraste_michelson,
        "Entropia": entropia
    }

def plotar_conversoes_cinza(imagens_dict, nome_arquivo):
    num_imagens = len(imagens_dict)
    fig, axes = plt.subplots(1, num_imagens, figsize=(num_imagens * 4, 5))
    fig.suptitle(f"Comparação de Métodos de Conversão: {nome_arquivo}", fontsize=16)
    
    if num_imagens == 1:
        axes = [axes]
        
    for ax, (nome, img) in zip(axes, imagens_dict.items()):
        ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        ax.set_title(nome)
        ax.axis('off')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.93])
    plt.show()

def medir_ruido(img):
    return np.std(img)

def medir_brilho_medio(img):
    return np.mean(img)

def medir_foco_laplaciano(img_cinza):
    return cv.Laplacian(img_cinza, cv.CV_64F).var()

def aplicar_filtros(img_cinza):
    img_gaussiano = cv.GaussianBlur(img_cinza, (5, 5), 0)
    img_mediano = cv.medianBlur(img_cinza, 5)
    img_bilateral = cv.bilateralFilter(img_cinza, 9, 75, 75)

    filtros = {
        'Gaussiano': img_gaussiano,
        'Mediano': img_mediano,
        'Bilateral': img_bilateral
    }
    return filtros

def avaliar_qualidade(df_resultados):
    ruido_orig = df_resultados.loc[df_resultados['Análise'] == 'Original', 'Ruído'].values[0]
    foco_orig = df_resultados.loc[df_resultados['Análise'] == 'Original', 'Foco'].values[0]

    df_filtros = df_resultados[df_resultados['Análise'] != 'Original'].copy()

    df_filtros['Redução Ruído'] = ((ruido_orig - df_filtros['Ruído']) / ruido_orig) * 100
    df_filtros['Perda Foco'] = ((foco_orig - df_filtros['Foco']) / foco_orig) * 100
    df_filtros['Pontuação'] = df_filtros['Redução Ruído'] - df_filtros['Perda Foco']

    melhor_filtro = df_filtros.loc[df_filtros['Pontuação'].idxmax()]

    relatorio = f"O filtro {melhor_filtro['Análise']} foi o mais eficaz."

    df_final = pd.concat([df_resultados.loc[df_resultados['Análise'] == 'Original'], df_filtros])
    df_final = df_final.set_index('Análise')
    df_final = df_final.fillna('-')

    return relatorio, df_final

imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""
imagem_processada_final = None
download_lock = False

# Célula 3: Upload de Múltiplas Imagens

In [ ]:
imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""

uploaded = files.upload()

if uploaded:
    arquivos_validos = 0
    for nome_arquivo, dados_arquivo in uploaded.items():
        extensao = nome_arquivo.split('.')[-1].lower()

        if extensao in ['jpg', 'jpeg', 'png']:
            try:
                buffer_imagem = np.frombuffer(dados_arquivo, np.uint8)
                imagem_bgr = cv.imdecode(buffer_imagem, cv.IMREAD_COLOR)

                imagens_carregadas[nome_arquivo] = imagem_bgr
                arquivos_validos += 1

            except Exception as e:
                pass
        else:
            pass
else:
    pass

# Célula 4: Visualização Inicial de Todas as Imagens

In [ ]:
if imagens_carregadas:
    for nome_arquivo, imagem_bgr in imagens_carregadas.items():
        imagem_rgb = cv.cvtColor(imagem_bgr, cv.COLOR_BGR2RGB)
        imagem_cinza = converter_para_cinza(imagem_bgr)

        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(imagem_rgb)
        axes[0].set_title(f'Original: {nome_arquivo}')
        axes[0].axis('off')

        axes[1].imshow(imagem_cinza, cmap='gray')
        axes[1].set_title('Convertida (Escala de Cinza)')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        altura, largura, _ = imagem_bgr.shape
else:
    pass

# Célula 5: Seleção da Imagem Ativa

In [ ]:
if imagens_carregadas:
    seletor_imagem = widgets.Dropdown(
        options=list(imagens_carregadas.keys()),
        description='Imagem Ativa:',
        disabled=False,
    )

    output_selecao = widgets.Output()

    def ao_selecionar_imagem(change):
        global imagem_selecionada_bgr, nome_arquivo_selecionado

        nome_arquivo_selecionado = change['new']
        imagem_selecionada_bgr = imagens_carregadas[nome_arquivo_selecionado]

        with output_selecao:
            clear_output(wait=True)
            imagem_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            plt.figure(figsize=(7, 5))
            plt.imshow(imagem_rgb)
            plt.title(f"Visualizando: {nome_arquivo_selecionado}")
            plt.axis('off')
            plt.show()

    seletor_imagem.observe(ao_selecionar_imagem, names='value')

    display(seletor_imagem, output_selecao)
    ao_selecionar_imagem({'new': seletor_imagem.value})

else:
    pass

# Célula 6: MÓDULO 0 - Análise de Métodos de Conversão

In [ ]:
if imagem_selecionada_bgr is not None:
    imagens_convertidas = converter_para_cinza_varios_metodos(imagem_selecionada_bgr)
    plotar_conversoes_cinza(imagens_convertidas, nome_arquivo_selecionado)

    resultados_metricas = {}
    for nome, img_cinza in imagens_convertidas.items():
        metricas = calcular_metricas_conversao(img_cinza)
        resultados_metricas[nome] = metricas
    
    df_conversao = pd.DataFrame.from_dict(resultados_metricas, orient='index')
    
    display(HTML(df_conversao.style.format('{:.4f}').to_html()))
    
else:
    pass

# Célula 7: MÓDULO 1 - Transformações de Contraste e Manipulações

In [ ]:
if imagem_selecionada_bgr is not None:

    algoritmo_dropdown = widgets.Dropdown(
        options=['Nenhum', 'Logaritmo', 'Quadrado', 'Exponencial', 'Equalização de Histograma'],
        value='Nenhum',
        description='Transformação:',
        disabled=False,
    )

    inverter_checkbox = widgets.Checkbox(
        value=False,
        description='Inverter Cores (Negativo)',
        disabled=False
    )

    output_area = widgets.Output()

    def on_controls_change(change):
        global imagem_processada_final

        with output_area:
            clear_output(wait=True)

            img_cinza = converter_para_cinza(imagem_selecionada_bgr)

            metodo = algoritmo_dropdown.value
            img_transformada = aplicar_transformacao(img_cinza, metodo)

            if inverter_checkbox.value:
                img_transformada = inverter_cores(img_transformada)

            imagem_processada_final = img_transformada

            imagem_original_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            fig, axes = plt.subplots(1, 2, figsize=(14, 7))

            axes[0].imshow(img_transformada, cmap='gray')
            axes[0].set_title(f"Imagem Processada\n(Transformação: {metodo})")
            axes[0].axis('off')

            axes[1].imshow(imagem_original_rgb)
            axes[1].set_title("Imagem Original (RGB)")
            axes[1].axis('off')

            plt.tight_layout()
            plt.show()

            brilho_antes = medir_brilho_medio(img_cinza)
            contraste_antes = medir_ruido(img_cinza)
            brilho_depois = medir_brilho_medio(img_transformada)
            contraste_depois = medir_ruido(img_transformada)

            df_metricas = pd.DataFrame({
                'Métrica': ['Brilho Médio', 'Contraste'],
                'Original (Cinza)': [brilho_antes, contraste_antes],
                'Processada': [brilho_depois, contraste_depois]
            })
            df_metricas = df_metricas.set_index('Métrica')

            display(df_metricas.style.format('{:.2f}'))

            plotar_histograma(img_transformada)

    algoritmo_dropdown.observe(on_controls_change, names='value')
    inverter_checkbox.observe(on_controls_change, names='value')

    display(widgets.VBox([
        widgets.HBox([algoritmo_dropdown, inverter_checkbox]),
        output_area
    ]))

    on_controls_change(None)

else:
    pass

# Célula 8: MÓDULO 2 - Análise de Filtros de Suavização

In [ ]:
if imagem_selecionada_bgr is not None:
    img_cinza_original = converter_para_cinza(imagem_selecionada_bgr)
    ruido_original = medir_ruido(img_cinza_original)
    foco_original = medir_foco_laplaciano(img_cinza_original)
    brilho_original = medir_brilho_medio(img_cinza_original)

    resultados = {
        'Análise': ['Original'],
        'Ruído': [ruido_original],
        'Foco': [foco_original],
        'Brilho Médio': [brilho_original]
    }

    imagens_filtradas = aplicar_filtros(img_cinza_original)

    for nome_filtro, img_filtrada in imagens_filtradas.items():
        ruido_filtro = medir_ruido(img_filtrada)
        foco_filtro = medir_foco_laplaciano(img_filtrada)
        brilho_filtro = medir_brilho_medio(img_filtrada)

        resultados['Análise'].append(nome_filtro)
        resultados['Ruído'].append(ruido_filtro)
        resultados['Foco'].append(foco_filtro)
        resultados['Brilho Médio'].append(brilho_filtro)

    df_resultados = pd.DataFrame(resultados)
    relatorio_final, df_tabela = avaliar_qualidade(df_resultados)

    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    fig.suptitle(f"Comparação de Filtros: {nome_arquivo_selecionado}", fontsize=16)

    axes[0, 0].imshow(img_cinza_original, cmap='gray')
    axes[0, 0].set_title(f"Original (Cinza)\nRuído: {ruido_original:.2f} | Foco: {foco_original:.2f}")
    axes[0, 0].axis('off')

    dados_gauss = df_tabela.loc['Gaussiano']
    axes[0, 1].imshow(imagens_filtradas['Gaussiano'], cmap='gray')
    axes[0, 1].set_title(f"Filtro Gaussiano\nRuído: {dados_gauss['Ruído']:.2f} | Foco: {dados_gauss['Foco']:.2f}")
    axes[0, 1].axis('off')

    dados_median = df_tabela.loc['Mediano']
    axes[1, 0].imshow(imagens_filtradas['Mediano'], cmap='gray')
    axes[1, 0].set_title(f"Filtro Mediano\nRuído: {dados_median['Ruído']:.2f} | Foco: {dados_median['Foco']:.2f}")
    axes[1, 0].axis('off')

    dados_bilat = df_tabela.loc['Bilateral']
    axes[1, 1].imshow(imagens_filtradas['Bilateral'], cmap='gray')
    axes[1, 1].set_title(f"Filtro Bilateral\nRuído: {dados_bilat['Ruído']:.2f} | Foco: {dados_bilat['Foco']:.2f}")
    axes[1, 1].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    format_percent = lambda v: f'{v:.2f}%' if isinstance(v, (int, float)) else v
    format_score = lambda v: f'{v:.2f}' if isinstance(v, (int, float)) else v

    display(df_tabela.style
        .map(lambda v: 'color: green' if isinstance(v, (int, float)) and v > 0 else ('color: red' if isinstance(v, (int, float)) and v < 0 else None),
             subset=['Redução Ruído', 'Perda Foco', 'Pontuação'])
        .format({
            'Ruído': '{:.2f}',
            'Foco': '{:.2f}',
            'Brilho Médio': '{:.2f}',
            'Redução Ruído': format_percent,
            'Perda Foco': format_percent,
            'Pontuação': format_score
        })
    )

    print(f"\n{relatorio_final}\n")

else:
    pass

# Célula 9: Download em Lote (ZIP)

In [ ]:
clear_output(wait=True)

DOWNLOAD_EM_ANDAMENTO = False

if 'imagens_carregadas' in globals() and imagens_carregadas:

    def iniciar_download_em_lote_zip(b):
        global DOWNLOAD_EM_ANDAMENTO

        if DOWNLOAD_EM_ANDAMENTO:
            return

        DOWNLOAD_EM_ANDAMENTO = True
        botao_download.disabled = True

        try:
            with output_download:
                clear_output(wait=True)

                escolha = escolha_download.value
                metodo_atual = 'Nenhum'
                inverter_atual = False

                if 'algoritmo_dropdown' in globals():
                    metodo_atual = algoritmo_dropdown.value
                if 'inverter_checkbox' in globals():
                    inverter_atual = inverter_checkbox.value

                nome_arquivo_zip = f"imagens_processadas_{int(time.time())}.zip"

                with zipfile.ZipFile(nome_arquivo_zip, 'w') as zf:
                    total = len(imagens_carregadas)
                    for i, (nome_arquivo, imagem_bgr) in enumerate(imagens_carregadas.items()):
                        imagem_final = None
                        sufixo_nome = ""

                        if escolha == 'Aplicar Transformação Atual (em todas)':
                            sufixo_nome = f"_{metodo_atual.lower().replace(' ', '_')}"
                            if inverter_atual:
                                sufixo_nome += "_invertido"
                            img_cinza = converter_para_cinza(imagem_bgr)
                            img_transformada = aplicar_transformacao(img_cinza, metodo_atual)
                            if inverter_atual:
                                img_transformada = inverter_cores(img_transformada)
                            imagem_final = img_transformada
                        else:
                            sufixo_nome = "_cinza"
                            imagem_final = converter_para_cinza(imagem_bgr)

                        if imagem_final is not None:
                            nome_base, extensao = os.path.splitext(nome_arquivo)
                            formato_saida = '.png' if extensao.lower() == '.png' else '.jpg'
                            nome_arquivo_saida = f"{nome_base}{sufixo_nome}{formato_saida}"

                            is_success, buffer_saida = cv.imencode(formato_saida, imagem_final)
                            if is_success:
                                zf.writestr(nome_arquivo_saida, buffer_saida.tobytes())
                            else:
                                pass

                files.download(nome_arquivo_zip)

        finally:
            DOWNLOAD_EM_ANDAMENTO = False
            botao_download.disabled = False

    label_download = widgets.Label(value="Opção de Download em Lote:")
    escolha_download = widgets.RadioButtons(
        options=['Aplicar Transformação Atual (em todas)', 'Converter para Cinza Padrão (em todas)'],
        description='',
        disabled=False
    )
    botao_download = widgets.Button(
        description="📦 Gerar e Baixar .ZIP",
        button_style='success',
        tooltip='Processar e baixar um arquivo ZIP com todas as imagens',
        icon='download'
    )
    output_download = widgets.Output()

    botao_download.on_click(iniciar_download_em_lote_zip)

    box_opcoes = widgets.VBox([label_download, escolha_download])
    display(box_opcoes, botao_download, output_download)

else:
    pass